In [3]:
import MDAnalysis as mda
import matplotlib.pyplot as plt
import math
import numpy as np

c:\Users\jeffs\Documents\Research_DrShen\MSD\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
graphite_water_psf = r'C:\Users\jeffs\Documents\Research_DrShen\MSD\msd_water_box\data_files\graphite_water.psf'
graphite_trajectory_dcd = r'C:\Users\jeffs\Documents\Research_DrShen\MSD\msd_water_box\data_files\output_per_40ps.dcd'
global_uni = mda.Universe(graphite_water_psf,graphite_trajectory_dcd)
global_uni.trajectory[0]
SPCE_uni = global_uni.select_atoms('resname SPCE')

c:\Users\jeffs\Documents\Research_DrShen\MSD\.venv\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [5]:
z_maxs = []

for frame in range(len(global_uni.trajectory)):
    global_uni.trajectory[frame]
    z_vals = []
    for atom in SPCE_uni.positions:
        z_vals.append(atom[2])
    z_maxs.append(max(z_vals))

average_z_max = sum(z_maxs)/len(z_maxs)
partition_count = 4
z_partitions = []

for i in range(int(global_uni.dimensions[2] / 10)):
    z_ceiling = i * 10
    if average_z_max < z_ceiling:
        for i in range(partition_count):
            z_partitions.append(z_ceiling / 4 * (i + 1))
        break

In [6]:
# Function to calculate MSD
def calculate_msd(position_dict):
    x_vals = np.array(position_dict['x_vals'])
    y_vals = np.array(position_dict['y_vals'])
    z_vals = np.array(position_dict['z_vals'])
    
    # Number of time points
    num_points = len(x_vals)
    
    # Initialize MSD array
    msd = np.zeros(num_points)
    
    # Calculate displacements and MSD
    for dt in range(1, num_points):
        displacements = []
        for t in range(num_points - dt):
            dx = x_vals[t + dt] - x_vals[t]
            dy = y_vals[t + dt] - y_vals[t]
            dz = z_vals[t + dt] - z_vals[t]
            displacement_squared = dx**2 + dy**2 + dz**2
            displacements.append(displacement_squared)
        
        msd[dt] = np.mean(displacements)
    
    return msd


In [40]:

msd_dict = {}

for atom_num in range(len(SPCE_uni)): # For each SPCE in our universe

    atom_part_movement = []
    atom_pos = {'x_vals':[],'y_vals':[],'z_vals':[]}

    for frame in range(len(global_uni.trajectory)): # For each frame in our animation

        global_uni.trajectory[frame] # Flip through each frame starting at 0 going to 500
        SPCE_positions = SPCE_uni.positions 
        current_z = SPCE_positions[atom_num][2]

        for i in range(len(z_partitions)): # For each partition we have

            if current_z < z_partitions[i]: # If our current z_value is below our z_partition[i], we label the partition in the ith partition
                atom_part_movement.append(i) 
                atom_pos['x_vals'].append(SPCE_positions[atom_num][0])
                atom_pos['y_vals'].append(SPCE_positions[atom_num][1])
                atom_pos['z_vals'].append(current_z)
                break
                # Append position data to dictionary for the specific atom we are in & break out of loop bc we know what partition we are in

            elif i == len(z_partitions): # If current_z was never less than z_partition[i], we know its above the partition. We then append said data and partition location
                atom_part_movement.append(i + 1)
                atom_pos['x_vals'].append(SPCE_positions[atom_num][0])
                atom_pos['y_vals'].append(SPCE_positions[atom_num][1])
                atom_pos['z_vals'].append(current_z)

    atom_n_switch_and_pos = {}
    ticker = -1

    for i in range(len(atom_pos['z_vals'])): # Looping through each possible position point for atom 1 throughout the trajectory

        if i == 0 or atom_part_movement[i - 1] != atom_part_movement[i]:
            ticker += 1
            atom_n_switch_and_pos[ticker] = {'x_vals':[],'y_vals':[],'z_vals':[]}
        # If we havent created a struct yet or our current part changed from the last part, create a struct

        atom_n_switch_and_pos[ticker]['x_vals'].append(atom_pos['x_vals'][i])
        atom_n_switch_and_pos[ticker]['y_vals'].append(atom_pos['y_vals'][i])
        atom_n_switch_and_pos[ticker]['z_vals'].append(atom_pos['z_vals'][i])
        # Appends positions to correct structure location

    msd_dict[atom_num] = {}
    for key in atom_n_switch_and_pos.keys(): # For each partition switch n atom did
        msd_dict[atom_num][key] = calculate_msd(atom_n_switch_and_pos[key])
    # Stores MSD data in msd_dict for each atom & each partition shift



100
